<a href="https://colab.research.google.com/github/KateT840/M-computacionales-1/blob/main/Parcial_4_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as spo
from scipy.stats import poisson, norm

In [ ]:
#Tomando desiciones (Waze)
#Punto 1 y 2
mu1= np.array([2.0, 2.1, 1.8, 2.0])
sigma1= np.array([0.20, 0.25, 0.15, 0.20])

mu2= np.array([2.0, 2.4, 2.0, 2.2])
sigma2= np.array([0.20, 0.30, 0.20, 0.25])

D1 = np.array([150.0, 160.0, 140.0, 155.0])
D2 = np.array([150.0, 160.0, 140.0, 155.0])
D_total = D1 + D2

N = 10**5

In [ ]:
def generar_tiempos(mu1, sigma1, mu2, sigma2, N):
    T = np.zeros((len(mu1), N))
    for i in range(len(mu1)):
        T1_i = norm.rvs(loc=mu1[i], scale=sigma1[i], size=N)
        T2_i = norm.rvs(loc=mu2[i], scale=sigma2[i], size=N)
        T[i, :] = T1_i + T2_i
    return T

In [ ]:
def calcular_velocidades(D_total, T):
    V = np.zeros_like(T)
    for i in range(len(D_total)):
        V[i, :] = D_total[i] / T[i, :]
    return V

In [ ]:
def calcular_probabilidad_evento(V):
    max_indices = np.argmax(V, axis=0)
    probs = np.array([np.mean(max_indices == i) for i in range(V.shape[0])])
    return probs

In [ ]:
T = generar_tiempos(mu1, sigma1, mu2, sigma2, N)
V = calcular_velocidades(D_total, T)
prob_prior = calcular_probabilidad_evento(V)

In [ ]:
#Punto 3
v_obs = np.array([70, 80, 50, 60])

In [ ]:
def calcular_verosimilitud(V, v_obs):
  mu_V = np.mean(V, axis=1)
  sigma_V = np.std(V, axis=1)
  L = norm.pdf(v_obs, loc=mu_V, scale=sigma_V)

  return mu_V, sigma_V, L

In [ ]:
mu_V, sigma_V, L = calcular_verosimilitud(V, v_obs)

In [ ]:
#Punto 4
def actualizacion_bayesiana(prob_prior, L):
  denominador = prob_prior * L
  prob_posterior = denominador/np.sum(denominador)
  return prob_posterior


In [ ]:
posterior = actualizacion_bayesiana(prob_prior, L)
posterior

array([7.45489350e-01, 2.11823851e-01, 6.82430491e-06, 4.26799751e-02])

In [ ]:
np.sum(actualizacion_bayesiana(prob_prior, L))

np.float64(0.9999999999999999)

In [ ]:
np.isclose(np.sum(posterior), 1.0)

np.True_

In [ ]:
tabla = pd.DataFrame({
    "Ruta": [f"Ruta {i+1}" for i in range(4)],
    "μV,i": mu_V,
    "σV,i": sigma_V,
    "Prior P(i)": prob_prior,
    "vobs,i": v_obs,
    "Likelihood Lᵢ": L,
    "Posterior P(i | vobs)": posterior
})

# Redondear para presentación bonita
tabla = tabla.round(4)
tabla

,Ruta,"μV,i","σV,i",Prior P(i),"vobs,i",Likelihood Lᵢ,Posterior P(i | vobs)
0,Ruta 1,75.3703,5.4245,0.3382,70,0.0451,0.7455
1,Ruta 2,71.6549,6.3849,0.1628,80,0.0266,0.2118
2,Ruta 3,73.9992,4.9380,0.2324,50,0.0000,0.0000
3,Ruta 4,74.2592,5.7732,0.2666,60,0.0033,0.0427


In [ ]:
#La Ruta 1 tiene la mayor probabilidad posterior: P(Ruta 1 | vobs) ≈ 0.7455
#Antes de observar vobs, la probabilidad de que la Ruta 1 fuera la mejor era de solo 0.3382
#Luego de aplicar el teorema de Bayes y considerar la observación de velocidad (70 km/h), esta probabilidad aumentó drásticamente a 0.7455.
